# Lezione 31 — TF-IDF e Text Mining

## Obiettivi di Apprendimento

Al termine di questa lezione sarai in grado di:

1. **Comprendere** i limiti del semplice Bag of Words e perché serve TF-IDF
2. **Calcolare** TF, IDF e TF-IDF sia manualmente che con sklearn
3. **Applicare** TF-IDF per classificazione e similarity testuale
4. **Interpretare** i pesi TF-IDF per capire quali parole sono importanti
5. **Configurare** TfidfVectorizer per diversi casi d'uso

## Importanza per il Data Analyst

TF-IDF è la tecnica più usata nel text mining classico perché:
- Pesa le parole per **importanza reale** nel corpus
- Penalizza parole troppo comuni (stopwords implicite)
- Migliora le prestazioni di classificazione rispetto al semplice BoW
- È alla base di motori di ricerca e sistemi di recommendation

## Posizione nel Percorso

```
BLOCCO 4: AI & NLP
├── Lezione 29: Fondamenti AI ✓
├── Lezione 30: Rappresentare il Testo ✓
├── Lezione 31: TF-IDF e Text Mining ◄── SEI QUI
├── Lezione 32: Sentiment Analysis
└── ...
```

---

# 1. Teoria Concettuale

## 1.1 Il Problema del Bag of Words

### Limite Fondamentale

Il Bag of Words tratta tutte le parole allo stesso modo: ogni occorrenza vale 1.

Ma non tutte le parole sono ugualmente **informative**:

| Parola | Frequenza | Informazione |
|--------|-----------|--------------|
| "il" | Altissima | Nulla (appare ovunque) |
| "prodotto" | Alta | Bassa (comune nel dominio) |
| "eccezionale" | Bassa | Alta (discriminante) |

### Esempio Concreto

**Corpus di recensioni prodotti**:
- Doc 1: "Il prodotto è buono"
- Doc 2: "Il prodotto è ottimo"
- Doc 3: "Il prodotto è pessimo"

Con BoW, la parola "il" e "prodotto" dominano la rappresentazione, ma sono **inutili** per distinguere i documenti. L'informazione reale è in "buono", "ottimo", "pessimo".

### Intuizione TF-IDF

L'idea è semplice:
- Una parola è importante se appare **spesso nel documento** (TF alto)
- Ma **raramente negli altri documenti** (IDF alto)

$$\text{Importanza} = \text{Frequenza locale} \times \text{Rarità globale}$$

---

## 1.2 Term Frequency (TF)

### Definizione

**TF (Term Frequency)** misura quanto spesso una parola appare in un documento.

Esistono diverse varianti:

| Variante | Formula | Caratteristica |
|----------|---------|----------------|
| **Raw count** | $tf(t,d) = f_{t,d}$ | Conteggio grezzo |
| **Boolean** | $tf(t,d) = 1$ se $t \in d$, else $0$ | Presenza/assenza |
| **Term frequency** | $tf(t,d) = \frac{f_{t,d}}{\sum_{t' \in d} f_{t',d}}$ | Normalizzata per lunghezza doc |
| **Log normalization** | $tf(t,d) = 1 + \log(f_{t,d})$ | Smorza differenze grandi |

### Esempio

Documento: "il gatto mangia il pesce il pesce"

| Parola | Raw count | Normalized TF |
|--------|-----------|---------------|
| il | 3 | 3/7 = 0.43 |
| gatto | 1 | 1/7 = 0.14 |
| mangia | 1 | 1/7 = 0.14 |
| pesce | 2 | 2/7 = 0.29 |

### Problema di TF da Solo

TF da solo non basta: la parola "il" ha TF massimo ma zero valore informativo. Serve un fattore che penalizzi parole comuni → IDF.

---

## 1.3 Inverse Document Frequency (IDF)

### Definizione

**IDF (Inverse Document Frequency)** misura quanto una parola è **rara** nel corpus.

$$idf(t, D) = \log\left(\frac{N}{df_t}\right)$$

Dove:
- $N$ = numero totale di documenti nel corpus
- $df_t$ = numero di documenti che contengono il termine $t$

### Intuizione

| Parola | In quanti doc? | IDF | Interpretazione |
|--------|----------------|-----|-----------------|
| "il" | 1000/1000 | log(1000/1000) = 0 | Inutile |
| "prodotto" | 800/1000 | log(1000/800) = 0.097 | Poco informativa |
| "eccezionale" | 10/1000 | log(1000/10) = 2.0 | Molto informativa |

### Variante sklearn

sklearn usa una formula leggermente diversa per evitare divisioni per zero:

$$idf(t) = \log\left(\frac{N + 1}{df_t + 1}\right) + 1$$

Questa formula:
- Aggiunge 1 al numeratore e denominatore (smoothing)
- Aggiunge 1 al risultato (garantisce IDF ≥ 1)

### Effetto dell'IDF

```
Parole comuni → IDF basso → peso finale basso
Parole rare   → IDF alto  → peso finale alto
```

---

## 1.4 TF-IDF Combinato

### Formula Finale

$$tfidf(t, d, D) = tf(t, d) \times idf(t, D)$$

In sklearn con le varianti di default:

$$tfidf(t, d) = f_{t,d} \times \left(\log\frac{N+1}{df_t+1} + 1\right)$$

Seguito da normalizzazione L2 del vettore risultante.

### Esempio Completo

**Corpus** (N=4 documenti):
- D1: "il gatto mangia"
- D2: "il cane mangia"
- D3: "il gatto dorme"
- D4: "il pesce nuota"

**Calcolo per D1**:

| Parola | tf(D1) | df | idf | tf-idf |
|--------|--------|-----|-----|--------|
| il | 1 | 4 | log(5/5)+1 = 1.0 | 1.0 |
| gatto | 1 | 2 | log(5/3)+1 = 1.51 | 1.51 |
| mangia | 1 | 2 | log(5/3)+1 = 1.51 | 1.51 |

Dopo normalizzazione L2: il vettore viene diviso per la sua norma.

### Proprietà Risultanti

1. **Parole comuni** (alto df) → basso tf-idf → poco peso
2. **Parole rare ma frequenti nel doc** → alto tf-idf → molto peso
3. **Parole uniche nel corpus** (df=1) → tf-idf massimo
4. **Vettori normalizzati** → confrontabili con cosine similarity

---

## 1.5 Cosine Similarity

### Il Problema della Similarità

Data la rappresentazione TF-IDF, come misuriamo quanto due documenti sono "simili"?

### Cosine Similarity

Misura l'angolo tra due vettori nello spazio:

$$\cos(\theta) = \frac{A \cdot B}{\|A\| \times \|B\|} = \frac{\sum_{i=1}^{n} A_i \times B_i}{\sqrt{\sum_{i=1}^{n} A_i^2} \times \sqrt{\sum_{i=1}^{n} B_i^2}}$$

### Interpretazione

| Valore | Significato |
|--------|-------------|
| 1.0 | Documenti identici (stessa direzione) |
| 0.0 | Documenti ortogonali (nessuna parola in comune) |
| -1.0 | Documenti opposti (raro con TF-IDF, che è ≥0) |

### Perché Cosine e non Euclidea?

| Metrica | Problema |
|---------|----------|
| Distanza Euclidea | Sensibile alla lunghezza del documento |
| Cosine Similarity | Indipendente dalla lunghezza (normalizzata) |

Un documento lungo e uno corto sullo stesso argomento:
- Distanza Euclidea: grandi differenze (magnitudini diverse)
- Cosine: alta similarità (stessa direzione)

### Visualizzazione

```
        │ B (doc lungo)
        │╱
        │
        │  A (doc corto)
        │╱
        └─────────────────

Angolo piccolo = alta cosine similarity
Anche se |B| >> |A|
```

---

# 2. Schema Mentale

## Mappa Decisionale: BoW vs TF-IDF

```
                    ┌─────────────────────────────────────┐
                    │   HAI TESTO DA VETTORIZZARE?       │
                    └──────────────────┬──────────────────┘
                                       │
              ┌────────────────────────┼────────────────────────┐
              │                        │                        │
              ▼                        ▼                        ▼
    ┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
    │ Solo conteggi   │    │ Pesatura per    │    │ Semantica       │
    │ (baseline)      │    │ importanza      │    │ avanzata        │
    └────────┬────────┘    └────────┬────────┘    └────────┬────────┘
             │                      │                      │
             ▼                      ▼                      ▼
    ┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
    │ CountVectorizer │    │ TfidfVectorizer │    │ Word Embeddings │
    │ (BoW)           │    │ (TF-IDF)        │    │ (Word2Vec,BERT) │
    └─────────────────┘    └─────────────────┘    └─────────────────┘
```

## Quando Usare TF-IDF

| Scenario | Usa TF-IDF? | Motivazione |
|----------|-------------|-------------|
| Classificazione documenti | ✓ Sì | Pesa parole discriminanti |
| Ricerca/Information retrieval | ✓ Sì | Caso d'uso originale |
| Document similarity | ✓ Sì | Con cosine similarity |
| Topic modeling | ✓ Spesso | Ma LDA può usare BoW |
| Sentiment analysis | ✓ Può aiutare | Ma BoW spesso ok |
| Sequenze (NER, POS) | ✗ No | Serve ordine, usa embeddings |

## Parametri Chiave TfidfVectorizer

| Parametro | Significato | Quando Usarlo |
|-----------|-------------|---------------|
| `sublinear_tf=True` | Usa 1+log(tf) | Corpus con termini molto ripetuti |
| `norm='l2'` | Normalizzazione L2 | Default, ok per similarity |
| `use_idf=True` | Applica IDF | Quasi sempre |
| `smooth_idf=True` | Smoothing IDF | Evita log(0), sempre on |
| `max_df=0.9` | Ignora in >90% docs | Stopwords automatiche |
| `min_df=2` | Ignora in <2 docs | Rimuovi typo e rare |

---

# 3. Notebook Dimostrativo

## TF-IDF in Pratica

In [ ]:
# ============================================================
# SETUP: Importazione librerie
# ============================================================

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
print("Librerie importate correttamente")

In [ ]:
# ============================================================
# CALCOLO MANUALE TF-IDF
# ============================================================
# Per capire esattamente cosa fa TfidfVectorizer

# Corpus di esempio
corpus = [
    "il gatto mangia il pesce",
    "il cane mangia la carne",
    "il gatto dorme sul divano",
    "il cane gioca nel parco"
]

print("=" * 60)
print("CALCOLO MANUALE TF-IDF")
print("=" * 60)

print("\nCORPUS:")
for i, doc in enumerate(corpus, 1):
    print(f"  D{i}: \"{doc}\"")

# Step 1: Costruiamo il vocabolario
all_words = set()
tokenized_docs = []
for doc in corpus:
    tokens = doc.lower().split()
    tokenized_docs.append(tokens)
    all_words.update(tokens)

vocab = sorted(list(all_words))
print(f"\nVocabolario ({len(vocab)} parole): {vocab}")

In [ ]:
# ============================================================
# Step 2: Calcolo TF (Term Frequency)
# ============================================================

N = len(corpus)  # Numero documenti

# Calcoliamo TF per ogni documento
print("\n" + "=" * 60)
print("STEP 2: TERM FREQUENCY (TF)")
print("=" * 60)

# Matrice TF (raw counts)
tf_matrix = np.zeros((N, len(vocab)))
word_to_idx = {w: i for i, w in enumerate(vocab)}

for doc_idx, tokens in enumerate(tokenized_docs):
    for token in tokens:
        tf_matrix[doc_idx, word_to_idx[token]] += 1

print("\nMatrice TF (conteggi grezzi):")
df_tf = pd.DataFrame(tf_matrix.astype(int), columns=vocab,
                     index=[f"D{i+1}" for i in range(N)])
print(df_tf)

In [ ]:
# ============================================================
# Step 3: Calcolo IDF (Inverse Document Frequency)
# ============================================================

print("\n" + "=" * 60)
print("STEP 3: INVERSE DOCUMENT FREQUENCY (IDF)")
print("=" * 60)

# df = document frequency (in quanti documenti appare ogni termine)
df_counts = np.sum(tf_matrix > 0, axis=0)

# IDF formula sklearn: log((N+1)/(df+1)) + 1
idf_values = np.log((N + 1) / (df_counts + 1)) + 1

print("\nDocument Frequency (df) e IDF per ogni parola:")
print("-" * 50)
idf_df = pd.DataFrame({
    'Parola': vocab,
    'df (in quanti doc)': df_counts.astype(int),
    'IDF': idf_values.round(3)
}).set_index('Parola')

print(idf_df)

print("\nOSSERVAZIONE:")
print("• 'il' appare in tutti i doc → IDF basso (1.0)")
print("• 'gatto','cane','mangia' in 2 doc → IDF medio")
print("• 'pesce','divano','parco'... in 1 doc → IDF alto")

In [ ]:
# ============================================================
# Step 4: Calcolo TF-IDF e Normalizzazione L2
# ============================================================

print("\n" + "=" * 60)
print("STEP 4: TF-IDF = TF × IDF")
print("=" * 60)

# TF-IDF = TF * IDF
tfidf_matrix = tf_matrix * idf_values

print("\nMatrice TF-IDF (prima di normalizzazione):")
df_tfidf_raw = pd.DataFrame(tfidf_matrix.round(3), columns=vocab,
                            index=[f"D{i+1}" for i in range(N)])
print(df_tfidf_raw)

# Normalizzazione L2 (ogni riga divisa per la sua norma)
norms = np.sqrt(np.sum(tfidf_matrix ** 2, axis=1, keepdims=True))
tfidf_normalized = tfidf_matrix / norms

print("\nMatrice TF-IDF (dopo normalizzazione L2):")
df_tfidf = pd.DataFrame(tfidf_normalized.round(3), columns=vocab,
                        index=[f"D{i+1}" for i in range(N)])
print(df_tfidf)

# Verifica: la norma di ogni riga deve essere 1
print("\nVerifica normalizzazione (norma di ogni riga):")
for i in range(N):
    norm = np.sqrt(np.sum(tfidf_normalized[i] ** 2))
    print(f"  ||D{i+1}|| = {norm:.6f}")

In [ ]:
# ============================================================
# CONFRONTO CON SKLEARN TfidfVectorizer
# ============================================================

print("\n" + "=" * 60)
print("VERIFICA: CONFRONTO CON sklearn TfidfVectorizer")
print("=" * 60)

# Usiamo TfidfVectorizer con stesse impostazioni
tfidf_vec = TfidfVectorizer()
X_sklearn = tfidf_vec.fit_transform(corpus)

# Convertiamo in DataFrame per confronto
vocab_sklearn = tfidf_vec.get_feature_names_out()
df_sklearn = pd.DataFrame(X_sklearn.toarray().round(3), 
                          columns=vocab_sklearn,
                          index=[f"D{i+1}" for i in range(N)])

print("\nMatrice TF-IDF da sklearn:")
print(df_sklearn)

# Confronto
print("\n" + "-" * 60)
print("Le due matrici sono identiche:", end=" ")

# Riordiniamo le colonne del nostro calcolo manuale
df_manual_sorted = df_tfidf[vocab_sklearn]
match = np.allclose(df_manual_sorted.values, df_sklearn.values, atol=1e-3)
print("✓ SÌ" if match else "✗ NO")

In [ ]:
# ============================================================
# COSINE SIMILARITY TRA DOCUMENTI
# ============================================================

print("\n" + "=" * 60)
print("COSINE SIMILARITY TRA DOCUMENTI")
print("=" * 60)

# Calcoliamo la matrice di similarità
sim_matrix = cosine_similarity(X_sklearn)

# Visualizziamo come DataFrame
df_sim = pd.DataFrame(sim_matrix.round(3),
                      columns=[f"D{i+1}" for i in range(N)],
                      index=[f"D{i+1}" for i in range(N)])

print("\nMatrice di Similarità Coseno:")
print(df_sim)

# Interpretazione
print("\n" + "-" * 60)
print("INTERPRETAZIONE:")
print("-" * 60)
print("""
• D1 ('gatto mangia pesce') vs D3 ('gatto dorme divano')
  Similarità: {:.3f} - condividono 'il', 'gatto'
  
• D2 ('cane mangia carne') vs D4 ('cane gioca parco')
  Similarità: {:.3f} - condividono 'il', 'cane'
  
• D1 vs D4: {:.3f} - condividono solo 'il' (bassa sim)
""".format(sim_matrix[0,2], sim_matrix[1,3], sim_matrix[0,3]))

In [ ]:
# ============================================================
# CONFRONTO BOW vs TF-IDF PER CLASSIFICAZIONE
# ============================================================

print("\n" + "=" * 60)
print("CONFRONTO BoW vs TF-IDF PER CLASSIFICAZIONE")
print("=" * 60)

# Dataset più grande per confronto significativo
recensioni = [
    # Positive (1)
    ("Prodotto eccezionale, lo consiglio a tutti", 1),
    ("Ottimo acquisto, sono molto soddisfatto", 1),
    ("Qualità superiore, prezzo giusto", 1),
    ("Spedizione veloce, prodotto perfetto", 1),
    ("Fantastico, supera le aspettative", 1),
    ("Molto buono, lo ricomprerei sicuramente", 1),
    ("Ottima qualità, consigliato vivamente", 1),
    ("Prodotto top, veramente eccellente", 1),
    ("Bellissimo prodotto, funziona benissimo", 1),
    ("Soddisfatto dell'acquisto, davvero ottimo", 1),
    ("Qualità prezzo imbattibile, consiglio", 1),
    ("Funziona perfettamente, acquisto azzeccato", 1),
    
    # Negative (0)
    ("Prodotto scadente, non funziona affatto", 0),
    ("Pessima qualità, soldi completamente sprecati", 0),
    ("Non lo consiglio, totalmente deludente", 0),
    ("Arrivato rotto, pessimo servizio clienti", 0),
    ("Qualità scarsa, non vale assolutamente il prezzo", 0),
    ("Terribile esperienza, da evitare assolutamente", 0),
    ("Prodotto difettoso, ho chiesto reso immediato", 0),
    ("Brutta esperienza, non comprate mai qui", 0),
    ("Deluso totalmente, prodotto completamente inutile", 0),
    ("Pessimo acquisto, assolutamente sconsigliato", 0),
    ("Non funziona come descritto, truffa", 0),
    ("Rottura dopo una settimana, qualità zero", 0)
]

texts = [r[0] for r in recensioni]
labels = np.array([r[1] for r in recensioni])

# Split
X_train_txt, X_test_txt, y_train, y_test = train_test_split(
    texts, labels, test_size=0.25, random_state=42, stratify=labels
)

print(f"Train: {len(X_train_txt)}, Test: {len(X_test_txt)}")

In [ ]:
# ============================================================
# CONFRONTO VETTORIZZAZIONI
# ============================================================

# Vectorizers
bow_vec = CountVectorizer()
tfidf_vec = TfidfVectorizer()

# Trasformazione
X_train_bow = bow_vec.fit_transform(X_train_txt)
X_test_bow = bow_vec.transform(X_test_txt)

X_train_tfidf = tfidf_vec.fit_transform(X_train_txt)
X_test_tfidf = tfidf_vec.transform(X_test_txt)

print("\n" + "-" * 60)
print("RISULTATI CLASSIFICAZIONE")
print("-" * 60)

# Testiamo con due classificatori
classifiers = {
    'Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42)
}

results = []

for clf_name, clf in classifiers.items():
    # BoW
    clf_bow = clf.__class__(**clf.get_params())
    clf_bow.fit(X_train_bow, y_train)
    acc_bow = accuracy_score(y_test, clf_bow.predict(X_test_bow))
    
    # TF-IDF
    clf_tfidf = clf.__class__(**clf.get_params())
    clf_tfidf.fit(X_train_tfidf, y_train)
    acc_tfidf = accuracy_score(y_test, clf_tfidf.predict(X_test_tfidf))
    
    results.append({
        'Classificatore': clf_name,
        'BoW': f"{acc_bow:.2%}",
        'TF-IDF': f"{acc_tfidf:.2%}",
        'Differenza': f"{(acc_tfidf - acc_bow)*100:+.1f}%"
    })

df_results = pd.DataFrame(results)
print("\n")
print(df_results.to_string(index=False))

print("""
NOTA: Con dataset piccoli le differenze possono variare.
Su dataset reali (1000+ documenti), TF-IDF tipicamente 
migliora le performance rispetto a BoW grezzo.
""")

In [ ]:
# ============================================================
# ANALISI: PAROLE PIÙ IMPORTANTI PER CLASSE
# ============================================================

print("\n" + "=" * 60)
print("PAROLE PIÙ IMPORTANTI PER CLASSE (TF-IDF)")
print("=" * 60)

# Alleniamo logistic regression su tutto il dataset
tfidf_full = TfidfVectorizer()
X_full = tfidf_full.fit_transform(texts)
clf_full = LogisticRegression(max_iter=1000, random_state=42)
clf_full.fit(X_full, labels)

# I coefficienti indicano l'importanza delle parole
feature_names = tfidf_full.get_feature_names_out()
coefficients = clf_full.coef_[0]

# Top parole positive e negative
top_k = 8
sorted_idx = np.argsort(coefficients)

print("\nTOP PAROLE INDICATIVE di NEGATIVO (coef < 0):")
print("-" * 40)
for idx in sorted_idx[:top_k]:
    print(f"  '{feature_names[idx]}': {coefficients[idx]:.3f}")

print("\nTOP PAROLE INDICATIVE di POSITIVO (coef > 0):")
print("-" * 40)
for idx in sorted_idx[-top_k:][::-1]:
    print(f"  '{feature_names[idx]}': {coefficients[idx]:.3f}")

---

# 4. Esercizi Svolti

## Esercizio 1: Calcolo TF-IDF Step-by-Step

**Problema**: Dato un mini-corpus di 3 documenti, calcola manualmente TF, IDF e TF-IDF per ogni termine, poi verifica con sklearn.

In [ ]:
# ============================================================
# ESERCIZIO 1 - SOLUZIONE
# Calcolo TF-IDF step-by-step
# ============================================================

print("=" * 70)
print("ESERCIZIO 1: CALCOLO TF-IDF STEP-BY-STEP")
print("=" * 70)

# Mini-corpus
mini_corpus = [
    "machine learning is great",
    "deep learning is a subset of machine learning",
    "neural networks power deep learning"
]

print("\nCORPUS:")
for i, doc in enumerate(mini_corpus, 1):
    print(f"  D{i}: \"{doc}\"")

N_docs = len(mini_corpus)

# Step 1: Tokenizzazione e vocabolario
print("\n" + "-" * 70)
print("STEP 1: VOCABOLARIO")
print("-" * 70)

tokenized = [doc.lower().split() for doc in mini_corpus]
vocab_set = set()
for tokens in tokenized:
    vocab_set.update(tokens)
vocabulary = sorted(list(vocab_set))

print(f"Vocabolario ({len(vocabulary)} termini): {vocabulary}")

# Step 2: Calcolo TF
print("\n" + "-" * 70)
print("STEP 2: TERM FREQUENCY (TF)")
print("-" * 70)

word_to_idx = {w: i for i, w in enumerate(vocabulary)}
tf_mat = np.zeros((N_docs, len(vocabulary)))

for doc_idx, tokens in enumerate(tokenized):
    for token in tokens:
        tf_mat[doc_idx, word_to_idx[token]] += 1

print("\nMatrice TF (conteggi):")
print(pd.DataFrame(tf_mat.astype(int), columns=vocabulary, 
                   index=[f"D{i}" for i in range(1, N_docs+1)]))

# Step 3: Calcolo IDF
print("\n" + "-" * 70)
print("STEP 3: INVERSE DOCUMENT FREQUENCY (IDF)")
print("-" * 70)

df_counts = np.sum(tf_mat > 0, axis=0)
idf_vals = np.log((N_docs + 1) / (df_counts + 1)) + 1

idf_table = pd.DataFrame({
    'Termine': vocabulary,
    'df': df_counts.astype(int),
    'IDF': idf_vals.round(4)
}).set_index('Termine')
print(idf_table)

# Step 4: TF-IDF
print("\n" + "-" * 70)
print("STEP 4: TF-IDF = TF × IDF (con normalizzazione L2)")
print("-" * 70)

tfidf_raw = tf_mat * idf_vals
norms = np.sqrt(np.sum(tfidf_raw ** 2, axis=1, keepdims=True))
tfidf_norm = tfidf_raw / norms

print("\nMatrice TF-IDF normalizzata:")
print(pd.DataFrame(tfidf_norm.round(4), columns=vocabulary,
                   index=[f"D{i}" for i in range(1, N_docs+1)]))

# Step 5: Verifica con sklearn
print("\n" + "-" * 70)
print("STEP 5: VERIFICA CON sklearn")
print("-" * 70)

sklearn_vec = TfidfVectorizer()
sklearn_tfidf = sklearn_vec.fit_transform(mini_corpus)
sklearn_vocab = sklearn_vec.get_feature_names_out()

# Riordiniamo le nostre colonne per confronto
my_tfidf_reordered = pd.DataFrame(tfidf_norm, columns=vocabulary)[sklearn_vocab].values
sklearn_arr = sklearn_tfidf.toarray()

match = np.allclose(my_tfidf_reordered, sklearn_arr, atol=1e-4)
print(f"\nCalcolo manuale = sklearn TfidfVectorizer? {'✓ SÌ' if match else '✗ NO'}")

---

## Esercizio 2: Sistema di Ricerca Documenti

**Problema**: Implementa un mini motore di ricerca che, data una query, restituisce i documenti più rilevanti ordinati per similarità TF-IDF.

In [ ]:
# ============================================================
# ESERCIZIO 2 - SOLUZIONE
# Mini motore di ricerca con TF-IDF
# ============================================================

print("=" * 70)
print("ESERCIZIO 2: MINI MOTORE DI RICERCA")
print("=" * 70)

# Corpus di documenti (articoli fittizi)
documents = [
    {"id": 1, "title": "Introduzione al Machine Learning", 
     "content": "Il machine learning è un campo dell'intelligenza artificiale che permette ai computer di apprendere dai dati senza essere esplicitamente programmati."},
    
    {"id": 2, "title": "Deep Learning e Reti Neurali",
     "content": "Il deep learning utilizza reti neurali profonde per apprendere rappresentazioni gerarchiche dei dati. È particolarmente efficace per immagini e testo."},
    
    {"id": 3, "title": "Python per Data Science",
     "content": "Python è il linguaggio più usato per data science e machine learning grazie a librerie come pandas, numpy e scikit-learn."},
    
    {"id": 4, "title": "Analisi dei Dati con Pandas",
     "content": "Pandas è una libreria Python fondamentale per l'analisi e manipolazione dei dati. Offre strutture dati efficienti come DataFrame."},
    
    {"id": 5, "title": "Intelligenza Artificiale nel Business",
     "content": "L'intelligenza artificiale sta trasformando il business. Le aziende usano AI per automazione, analisi predittiva e customer service."}
]

# Prepariamo il corpus
doc_contents = [d["content"] for d in documents]

# Costruiamo l'indice TF-IDF
search_vectorizer = TfidfVectorizer(
    max_features=1000,
    ngram_range=(1, 2),
    stop_words=None  # Per italiano custom
)
doc_vectors = search_vectorizer.fit_transform(doc_contents)

print(f"Indice costruito: {doc_vectors.shape[0]} documenti, {doc_vectors.shape[1]} features")

In [ ]:
# ============================================================
# FUNZIONE DI RICERCA
# ============================================================

def search(query: str, vectorizer, doc_vectors, documents, top_k: int = 3):
    """
    Cerca documenti rilevanti per una query.
    
    Parameters:
    -----------
    query : str
        Query di ricerca
    vectorizer : TfidfVectorizer
        Vectorizer addestrato sul corpus
    doc_vectors : sparse matrix
        Vettori TF-IDF dei documenti
    documents : list
        Lista di documenti originali
    top_k : int
        Numero di risultati da restituire
    
    Returns:
    --------
    list : Lista di (documento, score) ordinati per rilevanza
    """
    
    # Step 1: Vettorizza la query con lo stesso vectorizer
    query_vector = vectorizer.transform([query])
    
    # Step 2: Calcola similarità coseno con tutti i documenti
    similarities = cosine_similarity(query_vector, doc_vectors)[0]
    
    # Step 3: Ordina per similarità decrescente
    sorted_indices = np.argsort(similarities)[::-1]
    
    # Step 4: Restituisci top-k risultati
    results = []
    for idx in sorted_indices[:top_k]:
        if similarities[idx] > 0:  # Solo se c'è match
            results.append({
                'doc': documents[idx],
                'score': similarities[idx]
            })
    
    return results

# ── TEST DELLA RICERCA ──
print("\n" + "-" * 70)
print("TEST RICERCA")
print("-" * 70)

queries = [
    "machine learning intelligenza artificiale",
    "python pandas dataframe",
    "reti neurali deep learning"
]

for query in queries:
    print(f"\n🔍 QUERY: \"{query}\"")
    print("-" * 50)
    
    results = search(query, search_vectorizer, doc_vectors, documents, top_k=3)
    
    if results:
        for rank, res in enumerate(results, 1):
            print(f"  {rank}. [{res['score']:.3f}] {res['doc']['title']}")
    else:
        print("  Nessun risultato trovato")

---

## Esercizio 3: Estrazione Keywords con TF-IDF

**Problema**: Usa TF-IDF per estrarre automaticamente le parole chiave più importanti da un documento rispetto a un corpus di riferimento.

In [ ]:
# ============================================================
# ESERCIZIO 3 - SOLUZIONE
# Estrazione keywords con TF-IDF
# ============================================================

print("=" * 70)
print("ESERCIZIO 3: ESTRAZIONE KEYWORDS CON TF-IDF")
print("=" * 70)

def extract_keywords(document: str, vectorizer, top_n: int = 5) -> list:
    """
    Estrae le top-n keywords da un documento basandosi sui pesi TF-IDF.
    
    Parameters:
    -----------
    document : str
        Documento da analizzare
    vectorizer : TfidfVectorizer
        Vectorizer già fit su un corpus di riferimento
    top_n : int
        Numero di keywords da estrarre
    
    Returns:
    --------
    list : Lista di (keyword, score)
    """
    
    # Vettorizza il documento
    doc_vector = vectorizer.transform([document])
    
    # Ottieni feature names e scores
    feature_names = vectorizer.get_feature_names_out()
    scores = doc_vector.toarray()[0]
    
    # Trova indici con score > 0, ordinati per score decrescente
    nonzero_idx = np.where(scores > 0)[0]
    sorted_idx = nonzero_idx[np.argsort(scores[nonzero_idx])[::-1]]
    
    # Estrai top-n
    keywords = []
    for idx in sorted_idx[:top_n]:
        keywords.append((feature_names[idx], scores[idx]))
    
    return keywords

# Corpus di riferimento per calcolare IDF
reference_corpus = [
    "L'apprendimento automatico è una branca dell'intelligenza artificiale",
    "Python è un linguaggio di programmazione versatile",
    "Il deep learning usa reti neurali profonde",
    "I dati sono fondamentali per il machine learning",
    "Gli algoritmi di clustering raggruppano dati simili",
    "La classificazione predice categorie discrete",
    "La regressione predice valori continui",
    "Feature engineering migliora le performance dei modelli"
]

# Fit vectorizer sul corpus di riferimento
kw_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
kw_vectorizer.fit(reference_corpus)

# Documenti da analizzare
test_docs = [
    "Il deep learning e le reti neurali profonde stanno rivoluzionando l'intelligenza artificiale, permettendo ai computer di apprendere rappresentazioni complesse dei dati.",
    
    "Python è il linguaggio preferito per data science grazie a librerie come pandas per la manipolazione dati e scikit-learn per il machine learning."
]

print("\nESTRAZIONE KEYWORDS:")
print("-" * 70)

for i, doc in enumerate(test_docs, 1):
    print(f"\nDOCUMENTO {i}:")
    print(f"\"{doc[:80]}...\"")
    
    keywords = extract_keywords(doc, kw_vectorizer, top_n=6)
    
    print(f"\nKeywords estratte (top 6):")
    for kw, score in keywords:
        print(f"  • '{kw}': {score:.4f}")

---

# 5. Conclusione Operativa

## Cosa Abbiamo Imparato

| Concetto | Implicazione Pratica |
|----------|---------------------|
| **TF (Term Frequency)** | Misura frequenza locale di un termine nel documento |
| **IDF (Inverse Doc Freq)** | Pesa la rarità del termine nel corpus (parole comuni → peso basso) |
| **TF-IDF** | Combina TF×IDF per dare peso a parole localmente frequenti ma globalmente rare |
| **Normalizzazione L2** | Permette confronto equo tra documenti di lunghezza diversa |
| **Cosine Similarity** | Metrica standard per similarità tra vettori TF-IDF |

## Vantaggi di TF-IDF rispetto a BoW

| Aspetto | BoW | TF-IDF |
|---------|-----|--------|
| Parole comuni | Peso uguale a tutte | Peso ridotto automaticamente |
| Parole discriminanti | Trattate come le altre | Peso maggiore |
| Classificazione | Baseline | Tipicamente migliore |
| Information Retrieval | Meno efficace | Standard di riferimento |

## Workflow Operativo

```
1. PREPROCESSING
   └── Tokenizzazione, pulizia (come per BoW)

2. VETTORIZZAZIONE TF-IDF
   ├── TfidfVectorizer con parametri appropriati
   ├── fit_transform su train
   └── transform su test/query

3. APPLICAZIONE
   ├── Classificazione → passa matrice a classifier
   ├── Similarity → cosine_similarity tra vettori
   └── Keywords → ordina features per peso TF-IDF
```

## Errori Comuni da Evitare

1. ❌ Ignorare che IDF viene calcolato sul corpus di training
2. ❌ Non normalizzare quando si confrontano documenti di lunghezze diverse
3. ❌ Usare distanza Euclidea invece di cosine similarity
4. ❌ Dimenticare che parole nuove (OOV) hanno score 0

---

# 6. Bignami — Scheda di Riferimento Rapido

## Formule Chiave

**Term Frequency (raw):**
$$tf(t,d) = f_{t,d}$$

**Inverse Document Frequency (sklearn):**
$$idf(t) = \log\frac{N+1}{df_t+1} + 1$$

**TF-IDF:**
$$tfidf(t,d) = tf(t,d) \times idf(t)$$

**Cosine Similarity:**
$$\cos(\theta) = \frac{A \cdot B}{\|A\| \times \|B\|}$$

## Codice Essenziale

```python
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Configurazione tipica
vectorizer = TfidfVectorizer(
    max_features=10000,     # Limita vocabolario
    ngram_range=(1, 2),     # Unigram + bigram
    min_df=2,               # Ignora termini in < 2 docs
    max_df=0.95,            # Ignora termini in > 95% docs
    sublinear_tf=False,     # True per log(tf)
    norm='l2'               # Normalizzazione
)

# Fit e transform
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

# Similarità tra documenti
similarity = cosine_similarity(X_train[0:1], X_train)

# Keywords per documento
feature_names = vectorizer.get_feature_names_out()
tfidf_scores = X_train[0].toarray()[0]
top_indices = np.argsort(tfidf_scores)[::-1][:10]
keywords = [feature_names[i] for i in top_indices]
```

## Parametri TfidfVectorizer

| Parametro | Effetto |
|-----------|---------|
| `sublinear_tf=True` | Usa $1 + \log(tf)$ invece di tf grezzo |
| `smooth_idf=True` | Evita divisione per zero in IDF |
| `use_idf=True` | Applica IDF (False = solo TF normalizzato) |
| `norm='l2'` | Normalizza vettori a norma 1 |

---
*Fine Lezione 31 — TF-IDF e Text Mining*